In [1]:
pip install cobra


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 16.7 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 6.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 43.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Libraries needed for importing the model and defining the model

Loading the iJO1366.xml model as the object 'model'

In [2]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iJO1366.xml')

In [3]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [4]:
model.optimize() #initial optimal solution is 0.982

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,0.000000e+00
RNTR2c2,0.026541,2.775558e-17


Defining the metabolites

In [5]:
ethylene = Metabolite(id = 'eth_c', compartment='c', name='ethylene', formula='C2H4', charge=0)
ethylene_e = Metabolite(id = 'eth_e', compartment='e', name='ethylene_e', formula='C2H4', charge=0)
hydroxylarginine = Metabolite(id = 'hlarg_c', compartment='c', name='5-hydroxy-l-arginine', formula='C6H15N4O3', charge=1)
guanidine = Metabolite(id = 'guan_c', compartment='c', name='guanidine', formula='C1H6N3', charge=1)
ldelta1pyrroline5carboxylate = Metabolite(id = 'ld1p5carb_c', compartment='c', name='l-delta-1-pyrroline-5-carboxylate', formula='C5H7N1O2', charge=0)

Defining the reactions

In [6]:
Ethylene_forming_enzyme_main = Reaction('EFE_m')
Ethylene_forming_enzyme_side = Reaction('EFE_s')
Non_enzymatic_hlarg_degradation = Reaction('neHLARGdeg')
Ethylene_transport = Reaction('eth_tp')

Adding the metabolites to the reactions

In [7]:
Ethylene_transport.add_metabolites({ethylene: -1,
                            ethylene_e: 1,                      
                            })
Ethylene_forming_enzyme_side.add_metabolites({model.metabolites.akg_c: -1,              
                              model.metabolites.o2_c: -1,
                              model.metabolites.arg__L_c: -1,
                              hydroxylarginine: 1,
                              model.metabolites.co2_c: 1,
                              model.metabolites.succ_c: 1,
                             })

Non_enzymatic_hlarg_degradation.add_metabolites({hydroxylarginine: -1,
                              ldelta1pyrroline5carboxylate: 1,
                              guanidine: 1,
                              model.metabolites.h2o_c: 1,
                             })

Ethylene_forming_enzyme_main.add_metabolites({model.metabolites.akg_c: -1,              
                              model.metabolites.o2_c: -0.5,
                              #model.metabolites.arg__L_c: -1,  #in theory L-Arg is part of the reaction but is not changed              
                              ethylene: 1,
                              model.metabolites.co2_c: 3,
                              #model.metabolites.arg__L_c: 1,  #however, model does not accept L-arg as input and as output...
                              #model.metabolites.o2_c: 1,
                             })

Adding the reactions to the model

In [8]:
model.add_reactions([Ethylene_forming_enzyme_main])
model.add_reactions([Ethylene_forming_enzyme_side])
model.add_reactions([Non_enzymatic_hlarg_degradation])
model.add_reactions([Ethylene_transport])

Setting Reactions as demand reaction:
There are three types of reactions: Exchanges, Sinks and Demands
An exchange reaction is a reversible reaction that adds to or removes an extracellular metabolite from the extracellular compartment. A demand reaction is an irreversible reaction that consumes an intracellular metabolite. A sink is similar to an exchange but specifically for intracellular metabolites, i.e., a reversible reaction that adds or removes an intracellular metabolite.

In [9]:
model.add_boundary(model.metabolites.eth_c, type='demand')
model.add_boundary(model.metabolites.eth_e, type='exchange')
model.add_boundary(model.metabolites.hlarg_c, type='demand')
model.add_boundary(model.metabolites.guan_c, type='demand')
model.add_boundary(model.metabolites.ld1p5carb_c, type='demand')

Reaction identifier,DM_ld1p5carb_c
Name,l-delta-1-pyrroline-5-carboxylate demand
Memory address,0x7f7356576d40
Stoichiometry,ld1p5carb_c --> l-delta-1-pyrroline-5-carboxylate -->
GPR,
Lower bound,0
Upper bound,1000.0


Adding gene association:
AT1G05010 = locus tag found here: https://www.ncbi.nlm.nih.gov/gene?term=ethylene%20forming%20enzyme[gene]%20AND%20alive[prop]%20NOT%20newentry[gene]%20&sort=weight

In [10]:
Ethylene_forming_enzyme_main.gene_reaction_rule = '( AT1G05010 )'
Ethylene_forming_enzyme_main.genes

Ethylene_forming_enzyme_side.gene_reaction_rule = '( AT1G05010 )'
Ethylene_forming_enzyme_side.genes

frozenset({<Gene AT1G05010 at 0x7f7356577010>})

Itterate through model to check for successful implementation:

This is commented out because otherwise it always shows up when running %run iJ=1366_V0.ipynb in other files.

In [11]:
# Iterate through the the objects in the model

#print("Reactions")
#print("---------")
#for x in model.reactions:
#    print("%s : %s" % (x.id, x.reaction))

#print("")
#print("Metabolites")
#print("-----------")
#for x in model.metabolites:
#    print('%9s : %s' % (x.id, x.formula))

#print("")
#print("Genes")
#print("-----")
#for x in model.genes:
#    associated_ids = (i.id for i in x.reactions)
#    print("%s is associated with reactions: %s" %
#          (x.id, "{" + ", ".join(associated_ids) + "}"))

After adding the Metabolites and Reactions to the model, the effect of the modification on the model objective is tested.

In [12]:
model.optimize() #to check if adding metabolites and reactions had an effect on biomass -> it hasnt, good

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-0.296557
EX_co2_e,19.675223,0.000000
EX_cobalt2_e,-0.000025,0.000000
DM_4crsol_c,0.000219,0.000000
...,...,...
DM_eth_c,0.000000,0.000000
EX_eth_e,0.000000,0.000000
DM_hlarg_c,0.000000,0.000000
DM_guan_c,0.000000,0.000000


Checking the charge:

This is also commented out to prevent clogging of other files.

In [13]:
#print("Charge:")
#print("arg__L_c:")
#print(model.metabolites.arg__L_c.charge)
#print("succ_c:")
#print(model.metabolites.succ_c.charge)
#print("akg_c:")
#print(model.metabolites.akg_c.charge)
#print("O2_c:")
#print(model.metabolites.o2_c.charge)
#print("co2_c:")
#print(model.metabolites.co2_c.charge)
#print("hlarg_c:")
#print(model.metabolites.hlarg_c.charge)
#print("eth_c:")
#print(model.metabolites.eth_c.charge)

Create a  xml file with the modified model

In [14]:
from cobra.io import write_sbml_model

In [15]:
write_sbml_model(model, "modified_model.xml")